### Importing the necessary packages

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

from pathlib import Path
import requests
import os

# Setting to make numbers easier to read on display
pd.options.display.float_format = "{:20.3f}".format

### Downloading the Dataset

In [14]:
import os
import requests
import zipfile
import io
import pandas as pd

# Dataset URL
url = "https://archive.ics.uci.edu/static/public/502/online+retail+ii.zip"

# File paths
dataset_dir = "Dataset"
file_name = "online_retail_II.xlsx"
file_path = os.path.join(dataset_dir, file_name)

# Check if the Excel file already exists
if os.path.exists(file_path):
    print(f"Dataset already exists at: {file_path}")
else:
    # Download the zip file
    response = requests.get(url)
    if response.status_code == 200:
        # Create the dataset directory if it doesn't exist
        os.makedirs(dataset_dir, exist_ok=True)
        
        # Extract the zip file contents
        with zipfile.ZipFile(io.BytesIO(response.content)) as zip_file:
            zip_file.extractall(dataset_dir)
        print(f"Downloaded and extracted to: {dataset_dir}")
    else:
        print(f"Failed to download: {url}, HTTP Status Code: {response.status_code}")

Dataset already exists at: Dataset\online_retail_II.xlsx


### Data Exploration

In [23]:
dataset = pd.read_excel(file_path, sheet_name = 0)

print("First 5 rows of the dataset.\n")
dataset.head()

First 5 rows of the dataset.


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.950,13085.000,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.750,13085.000,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.750,13085.000,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.100,13085.000,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.250,13085.000,United Kingdom


In [18]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525461 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      525461 non-null  object        
 1   StockCode    525461 non-null  object        
 2   Description  522533 non-null  object        
 3   Quantity     525461 non-null  int64         
 4   InvoiceDate  525461 non-null  datetime64[ns]
 5   Price        525461 non-null  float64       
 6   Customer ID  417534 non-null  float64       
 7   Country      525461 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 32.1+ MB


In [20]:
dataset.describe()

,Quantity,InvoiceDate,Price,Customer ID
count,525461.000,525461,525461.000,417534.000
mean,10.338,2010-06-28 11:37:36.845017856,4.689,15360.645
min,-9600.000,2009-12-01 07:45:00,-53594.360,12346.000
25%,1.000,2010-03-21 12:20:00,1.250,13983.000
50%,3.000,2010-07-06 09:51:00,2.100,15311.000
75%,10.000,2010-10-15 12:45:00,4.210,16799.000
max,19152.000,2010-12-09 20:01:00,25111.090,18287.000
std,107.424,NaN,146.127,1680.811


In [22]:
dataset.describe(include ="O")

,Invoice,StockCode,Description,Country
count,525461,525461,522533,525461
unique,28816,4632,4681,40
top,537434,85123A,WHITE HANGING HEART T-LIGHT HOLDER,United Kingdom
freq,675,3516,3549,485852


In [28]:
(dataset["Customer ID"].isna().sum())

107927

In [29]:
# Number of NaN values in the "Customer ID" column.
print("Number of NaN values in the column 'Customer ID' is: {0}".format(dataset["Customer ID"].isna().sum()))

Number of NaN values in the column 'Customer ID' is: 107927


In [30]:
# First 5 rows of dataset in which the Customer ID is NaN.
dataset[dataset["Customer ID"].isna()].head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
263,489464,21733,85123a mixed,-96,2009-12-01 10:52:00,0.000,NaN,United Kingdom
283,489463,71477,short,-240,2009-12-01 10:52:00,0.000,NaN,United Kingdom
284,489467,85123A,21733 mixed,-192,2009-12-01 10:53:00,0.000,NaN,United Kingdom
470,489521,21646,NaN,-50,2009-12-01 11:44:00,0.000,NaN,United Kingdom
577,489525,85226C,BLUE PULL BACK RACING CAR,1,2009-12-01 11:49:00,0.550,NaN,United Kingdom
